# Lesson 4 - More APL operators: `∘` `@` `⌶`

Let's start with the [Jot](https://help.dyalog.com/latest/#Language/Symbols/Jot.htm) operator, `∘`.  `∘` comes from function composition, like how `f(g(x))` can be written `f∘g(x)` in mathematics. So, too, in APL, if `f` and `g` are functions, then `f∘g x` is the same as `f g x` (APL doesn't need parentheses for function application). This alone is, of course, not very interesting. However, APL also has dyadic (infix) functions: `A f∘g B` is `A f g B`.


Both of these are very important when writing tacit APL code. For example, if we want to write a function which adds its left argument to the reciprocal (monadic `÷`) of its right argument, it can be written as `f ← +∘÷`. 


The golden ratio (phi) can be calculated with the continued fraction

$\Large \phi = 1 + \frac{1}{1+\frac{1}{1+\frac{1}{1+\frac{1}{\ddots}}}}$

So $\phi$ is `1+÷1+÷1+÷…`. We can insert the same function between elements of a list with the `/` operator, for example,  

In [6]:
+/1 1 2 3 

7

In our case, we want to insert `…+÷…`, but that isn't a single function. However, we can use `+∘÷`: 

In [7]:
+∘÷/1 1 1 1 1 1 1 1 1 1

1.618181818181818181818181818181818

Let's increase our floating point precision a bit first,

In [10]:
⎕PP←34      ⍝ Increase print precision
⎕FR←1287    ⍝ 128-bit floats

`X⍴Y` reshapes `Y` into shape `X`:

In [11]:
+∘÷/1000⍴1  ⍝ A good approximation of phi.

1.618033988749894848204586834365638

`∘` also allows you to compose with argumnents. `g←f∘A` where `f` is a dyadic function and `A` an array (any data) gives `g`, a new monadic function which calculates `x f A`. Similarly, `g←A∘f` makes `g` a function which calculates `A f x`.

For _operators_ you can "curry" their right operand. So `WithTwo←∘2` is a new monadic operator which can in turn modify a dyadic function to become monadic (using 2 as its right argument). For example, `+ WithTwo 3` will give 5. 

In [14]:
WithTwo←∘2
+ WithTwo 3

5

This is especially useful with the `f⍣n` "power operator" which applies its `f` operand function `n` times. `twice←⍣2` is an operator which applies a function twice. For example, `2+twice 3` is 7. 

In [15]:
twice←⍣2
2+twice 3

7

`inv←⍣¯1` is an operator which will apply a function -1 times, i.e. applies the inverse of that function. 

Question:
> Do all functions have inverses?

No, but surprisingly many do. If you derive new functions tacitly using only operators and invertible functions, then the resulting function can also (generally) be inverted automatically. Even structural functions can be inverted: 

In [16]:
('x'∘,⍣¯1) 'x'∘, 'abc' 

abc

So what happened there is that we applied the function `'x'∘`, which prepends the letter `x`, and then we applied its inverse, which removes an `x` from the left side. The specific function `'x'∘`, is not hardcoded. Instead the interpreter has a bunch of rules which lets it determine the inverse of various compositions.

That's really all there is to say about `∘`. However, a warning is in place: `(f g)Y` is the same as `f∘g Y` which may fool you into thinking that `X(f g)Y` is the same as `X f∘g Y`. However, they are not the same!

A nice golfing trick using `∘` is having the left operand be `⊢`. This allows using a monadic function on the right argument while ignoring the left argument. 

The [At](https://help.dyalog.com/latest/#Language/Primitive%20Operators/At.htm) operator, `@`, does exactly what it says. What's on its left gets done at the position indicated by its right operand.

In [12]:
('X'@2 5) 'Hello'

HXllX

So we put an `X` at positions 2 and 5 (APL is 1-indexed by default – you can change to 0-indexing if you want). We can also give an array which matches the selected elements:

In [13]:
('XY'@2 5) 'Hello'

HXllY

So far, we've only used `@` to substitute elements. We can also use it to modify them: 

In [17]:
(-@2 5)10 20 30 40 50 60 

10 ¯20 30 40 ¯50 60

Here we applied the monadic function `-` (negate) at positions 2 and 5. We can do the same with a dyadic function, too:

In [18]:
7(+@2 5)10 20 30 40 50 60

10 27 30 40 57 60

So far, we have been using an array right operand. If we use a function right operand it gets applied to the right argument, and the result must be a Boolean mask instead of a list of indices.

In [19]:
⎕A ⍝ uppercase alphabet 
'x'@(∊∘⎕A)'Hello World' 

ABCDEFGHIJKLMNOPQRSTUVWXYZ

xello xorld

`∊` is membership, so the derived function `∊∘⎕A` gives a Boolean for where elements of the right (and only) argument are members of the uppercase alphabet: 

In [20]:
(∊∘⎕A)'Hello World'

1 0 0 0 0 0 1 0 0 0 0

which is then used as mask by `@` to determine where to substitute with `x`. See, for example [Goto the Nth Page](https://codegolf.stackexchange.com/questions/147318/goto-the-nth-page/147339#147339) where I use `@` twice.


[I-beam](https://help.dyalog.com/latest/#Language/Primitive%20Operators/I%20Beam.htm), `⌶`, is a special monadic operator (although it follows normal APL syntax) which uses a positive integer operand to select a functionality, typically from a range of system related services. Note that although documentation is provided for `⌶` functions, any service provided this way should be considered as "experimental" and subject to change – without notice – from one release to the next. 

One example is [Format Date-Time](https://help.dyalog.com/latest/#Language/I%20Beam%20Functions/Format%20Datetime.htm), `1200⌶`, which formats _Dyalog Date Numbers_ according to a set of pattern rules.

In [21]:
'%ISO%'(1200⌶)1⎕DT'J'

┌───────────────────┐
│2022-01-04T17:42:49│
└───────────────────┘